In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import *
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from flask import jsonify
import pickle


In [3]:
surf = pd.read_csv('/Users/victorialendof/Desktop/Jonathan/Galvanize/Capstones/Capstone3/surf_spots.csv')

In [4]:
link = surf.link

In [5]:
img_link = surf['img link']

In [6]:
soups = []
request = []

In [7]:
surf_hieght =[]
tide_hieght = []
wind = []
temp = []
water_temp = []


In [ ]:
for elem in link:
    request.append(requests.get(elem))

In [ ]:
for elem in request:
    soups.append(BeautifulSoup(elem.text, 'html.parser'))
    

In [ ]:
for elem in soups:
    surf_hieght.append(elem.find('span',{'class': 'quiver-surf-height'} ).text)
    tide_hieght.append(elem.find('span',{'class': 'quiver-reading'}).text)
    wind.append(elem.findAll('span',{'class': 'quiver-reading'})[1].text)
    water_temp.append(elem.find('div',{'class': 'quiver-water-temp'}).text)
    temp.append(elem.find('div',{'class': 'quiver-weather-stats'}).text)

In [ ]:
surf_hieght1 = []
for elem in surf_hieght:
    if len(elem) < 7:
        surf_hieght1.append(elem[2:-2])
    else:
        surf_hieght1.append(elem[3:-2])
surf_hieght2=[]
for elem in surf_hieght1:
    surf_hieght2.append(elem.replace('-',''))
        

surf_hieght = []

for elem in surf_hieght2:
    surf_hieght.append(int(elem))


In [ ]:
tide_hieght = map(lambda x: float(x[:-2]), tide_hieght)
wind = map(lambda x: int(x[:-3]), wind)
water_temp = map(lambda x: int(x[15:-3]), water_temp)
temp  = map(lambda x: int(x[7:-3]), temp )


In [ ]:
tide_hieght = list(tide_hieght)
wind = list(wind)
water_temp = list(water_temp)
temp = list(temp)

In [ ]:
surf['surf height ft'] = surf_hieght
surf['tide height ft'] = tide_hieght
surf['wind kts'] = wind
surf['water temp f'] = water_temp
surf['temp f'] = temp

In [ ]:
surf

In [ ]:
surf = surf.drop(columns=['link'])

In [ ]:
surf = surf.drop(columns=['img link'])

In [ ]:
surf_count = surf.groupby('region').count().sort_values(by= 'surfspot').reset_index()

In [ ]:
difficulty = surf.groupby('region').mean().sort_values(by= 'difficulty').reset_index()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
blah = surf_count['surfspot']
blahh = surf_count['region']
ax.barh(blahh,blah)
ax.set_title('Surfspots per County')
ax.set_ylabel('Counties')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.barh(difficulty['region'],difficulty['difficulty'])
ax.set_title('Average Difficulty by Region')
ax.set_ylabel('Counties')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.barh(difficulty['region'],difficulty['water temp f'])
ax.set_title('Average Temp by Region')
ax.set_ylabel('Counties')
plt.show()

In [ ]:
features = ['crowds',
            'difficulty',
            'surf height ft',
            'tide height ft',
            'wind kts',
            'water temp f',
            'temp f'    
]

In [ ]:
X = surf[features].values

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [ ]:
def cos_sim_recommendations(surfspot, region_name, n=5, region=None):
    index = surf.index[(surf['surfspot'] == surfspot) & (surf['region'] == region_name)][0]
    spots = X[index].reshape(1,-1)
    cs = cosine_similarity(spots, X)
    rec_index = np.argsort(cs)[0][::-1]
    ordered_df = surf.loc[rec_index]
    ordered_df = ordered_df.drop(index)
    
    if region:
        ordered_df = ordered_df[ordered_df['region'] == region]
    rec_df = ordered_df.head(n)
    orig_row = surf.loc[[index]].rename(lambda x: 'original')
    total = pd.concat((orig_row,rec_df))
    
    return total


In [ ]:
cos_sim_recommendations('PB Point', 'San Diego', )

In [ ]:
time = datetime.today()

In [ ]:
now =datetime.now()

In [ ]:
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
d3 = time.strftime("%m/%d/%y")

In [ ]:
d3 = d3.replace('/','_')

In [ ]:
if int(dt_string[14:-3]) < 12:
    surf.to_csv(f'surf_{d3}am.csv',index=False)
else:
    surf.to_csv(f'surf_{d3}pm.csv',index=False)

In [468]:
link.to_pickle('links.pkl')

In [467]:
img_link.to_pickle('img_link.pkl')

In [522]:
link_df.to_pickle('link.pkl')